# Design an mRNA for eGFP.

Jiajie Xu 19Nov22

# Overview

## Business Problem

Design mRNA is becoming hotest topic ever since pandemic. Using target protein reverse translate to a mRNA is facing difficulty due to computational cost and principles in biology design. Optimization of mRNA sequence is highly necessary. Less binging can reduce expand speed and increase mRNA translation efficiency. It will help reduce dose amount per injection and achieve samilar immune system response. Also, it may help reduce symptoms and uncomfort after vaccination.Therefore, there is a great commercial motivation for biopharma companies developing their own design algorithm. 

# Data Loading and cleaning

In [1]:
import numpy as np
import pandas as pd
import sys, re
from difflib import SequenceMatcher
import re
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False 
import os
# !pip install biopython
from Bio.Seq import Seq
from Bio import motifs
from collections import defaultdict
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from transformers import AutoTokenizer,TFBertModel
from sklearn.model_selection import train_test_split, KFold
import pydot
import matplotlib.pyplot as plt
import tensorflow as tf

## Import reference table for codon usage. 

For taget protein eFPG, it has been used in rattus norvegicus most often. Thus, here we decide to refer to Mus codon usage table to more close the the host live enviroment. 

In [2]:
df = pd.read_excel('codon_usage_rattus.xlsx')

In [3]:
display(df)

,Codon,amino acid,frequency
0,UUU,F,17.6
1,UUC,F,24.7
2,UUA,L,5.0
3,UUG,L,12.2
4,CUU,L,11.1
...,...,...,...
59,AGG,R,10.7
60,GGU,G,12.6
61,GGC,G,24.2
62,GGA,G,16.3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Codon       64 non-null     object 
 1   amino acid  64 non-null     object 
 2   frequency   64 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.6+ KB


In [5]:
df.sort_values(by=['amino acid'])

,Codon,amino acid,frequency
50,UGA,*,1.2
35,UAG,*,0.5
34,UAA,*,0.9
31,GCG,A,6.9
30,GCA,A,13.7
...,...,...,...
14,GUA,V,6.4
12,GUU,V,10.6
51,UGG,W,12.3
32,UAU,Y,11.6


In [6]:
#find most often use codon for each amino acid
df_select = df.groupby('amino acid').apply(lambda df: df.loc[df.frequency.idxmax()])

In [7]:
df_select.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, * to Y
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Codon       21 non-null     object 
 1   amino acid  21 non-null     object 
 2   frequency   21 non-null     float64
dtypes: float64(1), object(2)
memory usage: 672.0+ bytes


In [8]:
df_select['Codon']

amino acid
*    UGA
A    GCC
C    UGC
D    GAC
E    GAG
F    UUC
G    GGC
H    CAC
I    AUC
K    AAG
L    CUG
M    AUG
N    AAC
P    CCC
Q    CAG
R    CGC
S    UCC
T    ACC
V    GUG
W    UGG
Y    UAC
Name: Codon, dtype: object

In [9]:
# create both translation and reverse translation dictionary
codon_dict = df_select.set_index('amino acid').to_dict()['Codon']
rev_codon_dict = df.set_index('Codon').to_dict()['amino acid']

## Import eFPG sequence from vendor product specification

In [10]:
# mRNA sequence from https://www.trilinkbiotech.com/cleancap-egfp-mrna.html
online = 'AUGGUGAGCAAGGGCGAGGAGCUGUUCACCGGGGUGGUGCCCAUCCUGGUCGAGCUGGACGGCGACGUAAACGGCCACAAGUUCAGCGUGUCCGGCGAGGGCGAGGGCGAUGCCACCUACGGCAAGCUGACCCUGAAGUUCAUCUGCACCACCGGCAAGCUGCCCGUGCCCUGGCCCACCCUCGUGACCACCCUGACCUACGGCGUGCAGUGCUUCAGCCGCUACCCCGACCACAUGAAGCAGCACGACUUCUUCAAGUCCGCCAUGCCCGAAGGCUACGUCCAGGAGCGCACCAUCUUCUUCAAGGACGACGGCAACUACAAGACCCGCGCCGAGGUGAAGUUCGAGGGCGACACCCUGGUGAACCGCAUCGAGCUGAAGGGCAUCGACUUCAAGGAGGACGGCAACAUCCUGGGGCACAAGCUGGAGUACAACUACAACAGCCACAACGUCUAUAUCAUGGCCGACAAGCAGAAGAACGGCAUCAAGGUGAACUUCAAGAUCCGCCACAACAUCGAGGACGGCAGCGUGCAGCUCGCCGACCACUACCAGCAGAACACCCCCAUCGGCGACGGCCCCGUGCUGCUGCCCGACAACCACUACCUGAGCACCCAGUCCGCCCUGAGCAAAGACCCCAACGAGAAGCGCGAUCACAUGGUCCUGCUGGAGUUCGUGACCGCCGCCGGGAUCACUCUCGGCAUGGACGAGCUGUACAAGUAA'

In [11]:
rev_protein

NameError: name 'rev_protein' is not defined

In [ ]:
new_mRNA = ''
for i in range(len(rev_protein)):
    new_mRNA = new_mRNA + codon_dict.get(rev_protein[i])

In [ ]:
rev_protein = ''
res = [online[i:i+3] for i in range(0,len(online),3)]
for x in range(len(res)):
    rev_protein = rev_protein + rev_codon_dict.get(res[x])

In [ ]:
new_mRNA

## Verify the similarity of new design mRNA

In [ ]:
# calculate similarity of new design mRNA with original mRNA
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
similar('online','new_mRNA')

In [ ]:
similar('online','new_mRNA')

## Output target mRNA 

In [ ]:
# protrin sequence fetch from PDB online, use method to reverse translate mRNA
f=open('rcsb_pdb_6YLQ.fasta.','r')
lines=f.readlines()
protein = lines[1].rstrip('\n')
mRNA = ''
for i in range(len(protein)):
    mRNA = mRNA + codon_dict.get(protein[i])
mRNA

# Discussion and tryout for introduce modeling

In [ ]:
txt_file = open("protein.txt", "r")
file_content = txt_file.read()
file_content= file_content.replace('\n',"")
protein_txt = file_content.split(">lcl|")
protein_txt = protein_txt[1:]

txt_file = open("mRNA.txt", "r")
file_content = txt_file.read()
file_content= file_content.replace('\n',"")
mRNA_txt = file_content.split(">lcl|")
mRNA_txt = mRNA_txt[1:]

In [ ]:
def char_frequency(str1):
    dict = {}
    for n in str1:
        keys = dict.keys()
        if n in keys:
            dict[n] += 1
        else:
            dict[n] = 1
    return dict

In [ ]:
# protein_txt

In [ ]:
def data_clean(sequence, value):
    x = []
    y = []
    for i in range(len(sequence)):
        sequence[i] = sequence[i].split()      
        for j in range(len(sequence[i])):
            substring = "protein_id="
            if substring in sequence[i][j]:
                sequence[i][j]=sequence[i][j].replace("]","")
                arr1 = sequence[i][j].split("=")[1]
        arr2 = sequence[i][-1].split("]")[1]
        x.append(arr1)
        y.append(arr2)
        myDict = {"proteinID":x,
                   value:y}
    print(myDict)
    return(myDict)



In [ ]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10 due to long output
protein_dict = data_clean(protein_txt, "protein_seq")
df1 = pd.DataFrame(protein_dict)

In [ ]:
mRNA_dict = data_clean(mRNA_txt, "mRNA_seq")
df2 = pd.DataFrame(mRNA_dict)

In [ ]:
df_cleandata = pd.merge(df1, df2, on="proteinID")

In [ ]:
df_cleandata

In [ ]:
def percentageLetter(self, s: str, letter: str) -> int:
    n = len(s)
    ctr = 0
    for c in s:
        if c == letter:
            ctr += 1
    return floor(100 * ctr / n)

In [ ]:
df_cleandata

Drop less important value

In [ ]:
df_cleandata = df_cleandata.drop(["proteinID"], axis=1)

In [ ]:
# df_cleandata.isna().sum() 

## Adding more parameters for features need.

data filter for simplicity 

In [ ]:
#A_percent
df_cleandata['A'] = df_cleandata['mRNA_seq'].apply(lambda x: str(x).count("A")/len(str(x)))
# s.count(letter)/len(s)

#T_percent
df_cleandata['T'] = df_cleandata['mRNA_seq'].apply(lambda x: str(x).count("T")/len(str(x)))

#C_percent
df_cleandata['C'] = df_cleandata['mRNA_seq'].apply(lambda x: str(x).count("C")/len(str(x)))

#G_percent
df_cleandata['G'] = df_cleandata['mRNA_seq'].apply(lambda x: str(x).count("G")/len(str(x)))

#protein_length
df_cleandata['protein_len'] = df_cleandata['protein_seq'].apply(lambda x: len(str(x)))

#mRNA_length
df_cleandata['mRNA_len'] = df_cleandata['mRNA_seq'].apply(lambda x: len(str(x)))

In [ ]:
#cut-off range
X_cut = df_cleandata.mRNA_len.mean() - 1 * df_cleandata.mRNA_len.std()

In [ ]:
X_cut = int((X_cut//3)*3)

In [ ]:
X_cut

In [ ]:
#To reduce cal time, use 30 instead of 486
df_cleandata = df_cleandata[df_cleandata.mRNA_len > 30]
# df_cleandata = df_cleandata[df_cleandata.mRNA_len > X_cut]

In [ ]:
# df_cleandata

In [ ]:
# print(df_cleandata.mRNA_seq.str[:X_cut])
print(df_cleandata.mRNA_seq.str[:30])

In [ ]:
# df_cleandata.mRNA_seq = df_cleandata.mRNA_seq.str[:X_cut]
df_cleandata.mRNA_seq = df_cleandata.mRNA_seq.str[:30]

In [ ]:
df_cleandata

## Some testing for Position-specific Scoring Matrices model

In [ ]:
sites = df_cleandata['mRNA_seq'].apply(lambda x: Seq(x))
sites1 = [Seq('ACGTA'), Seq('TCGTA'),Seq('ACGAA')]
motif = motifs.create(sites1)
print(motif.counts)
motif.degenerate_consensus
W = motif.pssm
print(W)
k = len(motif.consensus)
motif.consensus
DNA = Seq('TCGACGTAGCGTACGATGCTAGCGATAGCTAGCTAGCTACGTGCTAGCTAGCTAG')
for i,S in W.search(DNA):
    print(i,S,DNA[i:i+k])
for i,S in W.search(DNA):
    if i<0:
        print(len(DNA) + i,S,DNA[i:i+k].reverse_complement(),'-')
    else:
        print(i,S,DNA[i:i+k], '+')
motif.pseudocounts = 1
W = motif.pssm
for i,S in W.search(DNA):
    if i<0:
        print(len(DNA) + i,S,DNA[i:i+k].reverse_complement(),'-')
    else:
        print(i,S,DNA[i:i+k], '+')
print(W)
type(W)

Using vector as a reference could be a better way, model will be able to including position infomation.